In [1]:
#install libraries
!pip install PyPDF2
!pip install pdfminer.six
!pip install reportlab

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 17.0 MB/s eta 0:00:00


In [2]:
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from PyPDF2 import PdfReader, PdfWriter

In [13]:
# testing aroung with writing to pdf

from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas

def create_pdf_with_wrapped_text(text, output_pdf_path, max_width):
    c = canvas.Canvas(output_pdf_path, pagesize=letter)

    # Split the text into lines to fit within the specified width
    lines = []
    current_line = ''
    for word in text.split():
        if c.stringWidth(current_line + ' ' + word) < max_width:
            current_line += ' ' + word
        else:
            lines.append(current_line.strip())
            current_line = word
    lines.append(current_line.strip())

    # Calculate the height of the text block
    line_height = 12  # Adjust as needed
    text_height = len(lines) * line_height

    # Draw the lines of text onto the PDF canvas
    y = 750  # Starting y-coordinate
    for line in lines:
        c.drawString(100, y, line)  # Adjust the coordinates as needed
        y -= line_height

    c.save()

# Example string of text
text = "This is a long string of text that needs to be wrapped to fit within the PDF width."

# Specify the path for the output PDF file
output_pdf_path = "output_pdf_file.pdf"

# Specify the maximum width for the text block
max_width = 400  # Adjust as needed

# Create the PDF with the wrapped text
create_pdf_with_wrapped_text(text, output_pdf_path, max_width)



In [14]:
#receives pdf through the input path
#return the first page number on which the phrase exists
#if phrase is not found: returns -1
def find_transcript_starting_page(input_pdf_path, phrase):
    with open(input_pdf_path, 'rb') as infile:
        reader = PdfReader(infile)

        for page_num in range(len(reader.pages)):
            page = reader.pages[page_num]
            text = page.extract_text()

            if phrase in text:
                return list(range(page_num, len(reader.pages)))

        return [-1]

In [12]:
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer

def extract_text_without_headers_footers(pdf_path, header_height, footer_height, page_height, starting_phrase):
    text = ""
    page_list = find_transcript_starting_page(pdf_path, starting_phrase)

    if page_list[0] == -1: return ""

    for page_layout in extract_pages(pdf_path, page_numbers = page_list):
        for element in page_layout:
            if isinstance(element, LTTextContainer):
                # Check if the text element is within header or footer region
                if not is_in_header_or_footer(element, header_height, footer_height, page_height):
                    text += element.get_text()
    return text

def is_in_header_or_footer(text_container, header_height, footer_height, page_height):
    # Define the y-coordinate range for the header and footer
    header_y_range = (page_height - header_height, page_height)
    footer_y_range = (0, footer_height)

    # Get the bounding box of the text container
    bbox = text_container.bbox

    # Check if the text container overlaps with the header or footer y-coordinate ranges
    # Here, we're checking if the top or bottom of the text container is within the header or footer range
    if bbox[1] >= header_y_range[0] or bbox[3] <= footer_y_range[1]:
        return True  # Text container is in header or footer
    else:
        return False  # Text container is not in header or footer

def write_text_to_pdf(text, output_pdf_path):
    c = canvas.Canvas(output_pdf_path, pagesize=letter)
    c.drawString(100, 750, text)  # Adjust the coordinates as needed
    c.save()

# Usage example
pdf_path = "test.pdf"
header_height = 50  # Example value, adjust as needed
footer_height = 50  # Example value, adjust as needed
page_height = 792   # Example value, adjust as needed
output_pdf_path = "output_pdf_file.pdf"  # Specify the path for the output PDF file
starting_phrase = "FINAL TRANSCRIPT"
extracted_text = extract_text_without_headers_footers(pdf_path, header_height, footer_height, page_height, starting_phrase)
# create_pdf_with_wrapped_text(extracted_text, output_pdf_path, 400)
with open('example2.txt', 'a') as f:
    f.write(extracted_text)
